<a href="https://colab.research.google.com/github/vtandroid/WebBT/blob/master/coolbg0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
client_id = "@@client_id" #@param {type:"raw"}


In [ ]:
import requests
MF_SERVER = "http://api-magicframe.automusic.win/"
def get_req(path):
  try:
    requests.get(MF_SERVER + path)
  except:
    try:
      requests.get(MF_SERVER + path)
    except Exception as e:
      print(str(e))
      pass
    pass
get_req("client/type/" + str(client_id)+"/2")

In [ ]:
!pip install moviepy==2.0.0.dev1
!pip install gizeh
!wget http://reupnet.info/tool/coolbg/magick
!chmod 777 magick
!cp -r magick /usr/bin/
!mkdir -p /u02 && cd /u02 && rm -rf *.zip &&  wget http://reupnet.info/tool/gbak/drive_config.zip && unzip -o drive_config.zip
!pip install gbak
!pip install coolbg
!pip install requests
!pip install python-slugify
import os 
os.environ["IMAGEMAGICK_BINARY"] = "/usr/bin/magick"
from PIL import Image
import numpy as np
import requests,time,json
from moviepy.editor import *
from bgeditor import BGEditor
from bgeditor.common.utils import upload_file
from slugify import slugify
import ntpath
get_req("client/type/" + str(client_id)+"/3") #install done


In [ ]:
import traceback
while True:
 
  url_log=MF_SERVER+"job/log"
  try:
    try:
      get_req("client/setw/"+str(client_id)+"/0")
      if "restart-1" in requests.get(MF_SERVER+"client/check-restart/"+str(client_id)).text:
        print("Stop.. wait Restart")
        get_req("client/setw/"+str(client_id)+"/2")
        break
      print("Get Job: "+str(client_id))
      obj=requests.get(MF_SERVER+"job/get/client_id/"+str(client_id)).json()
      if obj and "id" in obj:
        #start Job
        name_vid=slugify(obj['name'])

        os.system("rm -rf /tmp/download/*")
        os.system("rm -rf /tmp/coolbg_ffmpeg/*")
        os.system("rm -rf /tmp/results/*")
        get_req("client/setw/"+str(client_id)+"/1")
        list_comps= json.loads(obj['content'])
        bgtool= BGEditor()
        path_vid_orgi = bgtool.create_video_json(list_comps,obj['id'])
        print(path_vid_orgi)
        head_path, tail_path = ntpath.split(path_vid_orgi)
        extension=tail_path.split(".")[1]
        path_thumbnail=ntpath.join(head_path,name_vid+"-thumb.jpg")
        path_vid=ntpath.join(head_path,name_vid+"-video."+extension)
        os.rename(path_vid_orgi,path_vid)
        
        vid_done=VideoFileClip(path_vid, audio=False)
        vid_done.save_frame(path_thumbnail,t=obj['thumbnail_seconds'])
        vid_done.close()
        file_id= upload_file(path_vid)
        file_id_thumbnail=upload_file(path_thumbnail)
        print(file_id)
        bgtool.close()
        data_post={"id":obj['id'],"client_id":str(client_id),"success":1, "log": "done", "result":file_id+";;"+file_id_thumbnail}
        requests.post(url_log, data = data_post)
      else:
        print("No Job!!!!!")
        time.sleep(10)

    except Exception as e:
      if obj and "id" in obj: 
        data_post={"id":obj['id'],"client_id":str(client_id),"success":"0", "log": traceback.format_exc()}
        requests.post(url_log, data = data_post)
      time.sleep(5)
      pass

  except Exception as e2:
    print(str(e2))
    time.sleep(10)
    pass